# Import

In [ ]:
!pip install autograd

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import autograd.numpy as au
import scipy

from scipy.optimize import line_search
from autograd import grad, jacobian
from scipy.optimize import minimize

from plotly.subplots import make_subplots
import plotly.graph_objects as go

In [ ]:
NORM = np.linalg.norm

# Function Definition

In [ ]:
def func(x): 
    # Objective function (Branin function)
    return (x[1] - (5.1/(4*au.pi**2))*x[0]**2 + (5/au.pi)*x[0] - 6)**2 + 10*(1 - 1/(8*au.pi))*au.cos(x[0]) + 10
    

 # Gradient of the objective function
Df = grad(func)

In [ ]:
# Create a grid of x0 and x1 values
x0 = np.linspace(-5, 16, 100)
x1 = np.linspace(-5, 16, 100)
X0, X1 = np.meshgrid(x0, x1)
Z = func([X0, X1])

In [ ]:
contours=plt.contour(x0,
                     x1,
                     Z,
                     100,
                     cmap=plt.cm.gnuplot)

plt.clabel(contours,
           inline=1,
           fontsize=10)

plt.xlabel("$x_1$ ->")
plt.ylabel("$x_2$ ->")

plt.show()

# BFGS

In [ ]:
def BFGS(Xj, tol, alpha_1, alpha_2):
    x_history = [Xj.copy()]  # List to store the history of point updates
    Bf = np.eye(len(Xj))
    
    while True:
        Grad = Df(Xj)
        
        # Selection of the direction of the steepest descent
        delta = -Bf.dot(Grad) 
                
        start_point = Xj  # Start point for step length selection 
        
        alpha = line_search(f=func, myfprime=Df, xk=start_point, pk=delta, c1=alpha_1, c2=alpha_2)[0]  # Selecting the step length
        
        if alpha is not None:
            X = Xj + alpha * delta
        
        if np.linalg.norm(Df(X)) < tol:
            x_history.append(X.copy())
            return np.array(x_history)
        
        else:
            Dj = X - Xj
            Gj = Df(X) - Grad
            
            den = Dj.dot(Gj)
            num = Bf.dot(Gj)
            
            L = 1 + num.dot(Gj) / den
            M = np.outer(Dj, Dj) / den
            N = np.outer(Dj, num) / den
            O = np.outer(num, Dj) / den
            
            Delta = L * M - N - O
            Bf += Delta
            Xj = X  # Update to the new iterate
            x_history.append(X.copy())
    
    return np.array(x_history)


In [ ]:
history = BFGS(np.array([1.5, 7.75]), 10**-5, 10**-4, 0.25)
history

In [ ]:
# Create a 3D surface plot
startContours = -5
endContours = 100
sizeContours = 5


fig = go.Figure(data=[go.Scatter3d(x=history[:, 0],
                                   y=history[:, 1],
                                   z=func(history),
                                   mode='lines+markers',
                                   marker = dict(symbol = 'circle',
                                                 color = 'red',
                                                 size = 5))])



fig.add_trace(go.Surface(x=X0,
                         y=X1,
                         z=Z,
                         showscale=False,
                         contours=dict(z=dict(color = "limegreen",
                                              highlight = True,
                                              show = True,
                                               project=dict(x=True,
                                                           y=True,
                                                           z=False),
                                              size = sizeContours,
                                              start = startContours,
                                              end = endContours))))

fig.update_layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=dict(l=50, r=50, b=100, t=100, pad=4),
    scene=dict(
        xaxis_title="X Axis",  # Set X-axis title
        yaxis_title="Y Axis",  # Set Y-axis title
        zaxis_title="Z Axis",  # Set Z-axis title
    ),
    showlegend =False,
    template="plotly_dark"
)



fig.show()

In [ ]:
def func(x, y):
    # Objective function (Branin function)
    return (y - (5.1 / (4 * au.pi**2)) * x**2 + (5 / au.pi) * x - 6)**2 + 10 * (1 - 1 / (8 * au.pi)) * au.cos(x) + 10

# Animation

In [ ]:
# Initialize figure with subplots
fig = make_subplots(rows=1, cols=1, specs=[[{'type': 'surface'}]])

# Create a 3D surface plot with contours
fig.add_trace(go.Surface(x=X0, y=X1, z=Z, showscale=False,
                         contours=dict(z=dict(color="limegreen",
                                              show=True,
                                              highlight=True,
                                              project=dict(x=True, y=True,z=True),
                                              size=sizeContours,
                                              start=startContours, 
                                              end=endContours))),
              row=1, col=1)


# Initialize scatter plot data
# Add scatter plot trace to the figure
scatter_data = go.Scatter3d(x=[], y=[], z=[], mode='lines+markers', marker=dict(symbol='x', color='red'))
fig.add_trace(scatter_data, row=1, col=1)

# Define animation frames
frames = []
for i in range(len(history)):
    print("Frame-{}".format(i))
    frame_data = [go.Surface(x=X0,
                   y=X1,
                   z=Z,
                   contours=dict(z=dict(color="limegreen",
                                        show=True,
                                        highlight=True,
                                        project=dict(x=True, y=True,z=True),
                                        size=sizeContours, 
                                        start=startContours, 
                                        end=endContours))),# Include the surface in each frame
                  go.Scatter3d(x=history[:i, 0],
                     y=history[:i, 1],
                     z=func(history[:i, 0],history[:i, 1]),
                     mode='lines+markers',
                     marker = dict(symbol = 'circle',
                                   color = 'white',
                                   size = 5))]

    frame = go.Frame(data=frame_data, name=f'Frame {i}')
    frames.append(frame)

# Add frames to the animation
fig.frames = frames

# Create animation buttons
animation_buttons = [
    dict(label="Play",
         method="animate",
         args=[None, {"frame": {"duration": 500, "redraw": True}, "fromcurrent": True}]),
    dict(label="Pause",
         method="animate",
         args=[[None], {"frame": {"duration": 0, "redraw": True}, "mode": "immediate", "transition": {"duration": 0}}])
]

# Update layout with animation settings
fig.update_layout(updatemenus=[{"buttons": animation_buttons, "type": "buttons", "showactive": False}],
                  title="BFGS Animation")


fig.update_layout(
    autosize=False,
    width=1000,
    height=1000,
    margin=dict(l=50, r=50, b=100, t=100, pad=4),
    scene=dict(
        xaxis_title="X Axis",  # Set X-axis title
        yaxis_title="Y Axis",  # Set Y-axis title
        zaxis_title="Z Axis",  # Set Z-axis title
    ),
    showlegend =False,
    template = "plotly_dark"
)

# Show the figure
fig.show()


# Contours

In [ ]:
fig = go.Figure(data =
         go.Contour(
           z= Z,
           colorbar=dict(nticks=10, 
                         ticks='outside',
                         ticklen=5, 
                         tickwidth=1,
                         showticklabels=True,
                         tickangle=0, 
                         tickfont_size=12)
            ))

fig.add_trace(go.Scatter(x=history[:, 0], 
                         y=history[:, 1], 
                         mode='markers+lines', 
                         name='BFGS',
                         marker=dict(color='white')))

# Update layout to increase size
fig.update_layout(
    width=1000,  # Set the width of the figure
    height=1000,  # Set the height of the figure
    title='Contour Plot',  # Add a title to the plot
    xaxis_title="X Axis",  # Label the x-axis
    yaxis_title="Y Axis",  # Label the y-axis
    template="plotly_dark"  # Use a dark theme for the plot
)


fig.show()

In [ ]:
def func(x): # Objective function
    return (x[1] - (5.1/(4*au.pi**2))*x[0]**2 + (5/au.pi)*x[0] - 6)**2 + 10*(1 - 1/(8*au.pi))*au.cos(x[0]) + 10

Df = grad(func) # Gradient of the objective function

res=minimize(fun=func,
             x0=np.array([1.5, 7.75]), 
             jac=Df, 
             method='BFGS',
             options={'gtol':10**-5,
                      'disp':True, 
                      'return_all':True})

In [ ]:
res.allvecs

In [ ]:
history